<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400&display=swap" rel="stylesheet">
<div style="font-family: 'Inter'; font-size: 24px; color: #749857;"><B>🍏 Show Up</b> for Health</div>
<div style="font-family: 'Inter'; font-size: 14px; color: #7a7979;"><B>Predicting Missed Apppointment in Primary Care</b> - Brompton Health PCN</div>

[janduplessis883](https://github.com/janduplessis883)<BR>
[AlexAlexRose](https://github.com/AlexAlexRose)<BR>
[FabySp](https://github.com/FabySp)<BR>
[mogleyza](https://github.com/mogleyza)

# FabySp | API Testing 

### Importing Libraries

In [1]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

### Params

In [2]:
RAW_DATA = '~/code/janduplessis883/data-showup/data/raw-data/'
WEATHER_DATA = '~/code/janduplessis883/data-showup/data/weather/weather.csv'
IMD_DATA = '~/code/janduplessis883/data-showup/data/imd-master/imd_master.csv'

OUTPUT_DATA = '~/code/janduplessis883/data-showup/data/output-data/'

In [3]:
# Importing the dataset
df = pd.read_csv('~/code/janduplessis883/data-showup/data/raw-data/ECS/ECS_APP1.csv')
df.head()

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842
1,12-May-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589
2,17-May-22,05-Jul-22,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135
3,31-May-22,07-Jul-22,Finished,11:00 - 11:59,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,19579593
4,07-Jun-22,05-Jul-22,Did Not Attend,11:00 - 11:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,60020944


In [4]:
# Making a copy of the dataset to play with
df_copy = df.copy()
df_copy.head(10)

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842
1,12-May-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589
2,17-May-22,05-Jul-22,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135
3,31-May-22,07-Jul-22,Finished,11:00 - 11:59,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,19579593
4,07-Jun-22,05-Jul-22,Did Not Attend,11:00 - 11:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,60020944
5,10-Jun-22,07-Jul-22,Finished,09:00 - 09:59,"FRANKSON, Joyce (Ms)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19581430
6,10-Jun-22,18-Jul-22,Finished,09:00 - 09:59,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,17407665
7,13-Jun-22,25-Jul-22,Finished,10:00 - 10:59,"ADIB, Burhan (Dr.)","EMILIANI, Orietta (Dr)",Earls Court Surgery GP,19579833
8,14-Jun-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19581485
9,14-Jun-22,05-Jul-22,Finished,18:00 - 18:59,"ADIB, Burhan (Dr.)","ADIB, Burhan (Dr.)",Earls Court Surgery Extended Hours GP,19579335


In [5]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9374 entries, 0 to 9373
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Appointment booked date  9374 non-null   object
 1   Appointment date         9374 non-null   object
 2   Appointment status       9374 non-null   object
 3   Appointment time         9374 non-null   object
 4   Booked by                9374 non-null   object
 5   Clinician                9374 non-null   object
 6   Rota type                9374 non-null   object
 7   Patient ID               9374 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 586.0+ KB


In [21]:
# Working on the appointment time to have 1 time only
split_df = df_copy['Appointment time'].str.split(' - ', expand=True)  # Split the timeframes
df_copy['Appointment time'] = split_df[0]  # Keep 'time1' and remove 'time2'

df_copy.head()

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID
0,10-May-22,14-Jul-22,Finished,10:00,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842
1,12-May-22,05-Jul-22,Finished,09:00,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589
2,17-May-22,05-Jul-22,Finished,10:00,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135
3,31-May-22,07-Jul-22,Finished,11:00,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,19579593
4,07-Jun-22,05-Jul-22,Did Not Attend,11:00,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,60020944


In [22]:

# Merge 'date' and 'time' columns
df_copy['app datetime'] = pd.to_datetime(df_copy['Appointment date'] + ' ' + df_copy['Appointment time'])
df_copy.head()

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime
0,10-May-22,14-Jul-22,Finished,10:00,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00
1,12-May-22,05-Jul-22,Finished,09:00,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589,2022-07-05 09:00:00
2,17-May-22,05-Jul-22,Finished,10:00,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135,2022-07-05 10:00:00
3,31-May-22,07-Jul-22,Finished,11:00,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,19579593,2022-07-07 11:00:00
4,07-Jun-22,05-Jul-22,Did Not Attend,11:00,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,60020944,2022-07-05 11:00:00


In [13]:
df_weather = pd.read_csv('~/code/janduplessis883/data-showup/data/weather/weather.csv')
df_weather.head(-10)

,app datetime,temp,precipitation
0,2018-01-01T00:00,6.9,0.0
1,2018-01-01T01:00,6.2,0.0
2,2018-01-01T02:00,5.8,0.0
3,2018-01-01T03:00,5.6,0.0
4,2018-01-01T04:00,5.4,0.0
...,...,...,...
49281,2023-08-16T09:00,19.4,0.0
49282,2023-08-16T10:00,21.0,0.0
49283,2023-08-16T11:00,22.1,0.2
49284,2023-08-16T12:00,22.4,0.0


In [9]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49296 entries, 0 to 49295
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   app datetime   49296 non-null  object 
 1   temp           49296 non-null  float64
 2   precipitation  49296 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.1+ MB


In [23]:
# merging the app datetime columns from two different csv files
df_weather['app datetime'] = pd.to_datetime(df_weather['app datetime'])

merged_df = pd.merge(df_copy, df_weather, on='app datetime')
merged_df.head(-10)

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation
0,10-May-22,14-Jul-22,Finished,10:00,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0
1,05-Jul-22,14-Jul-22,Finished,10:00,"JAYATISSA, Rekha (Mrs)","EMILIANI, Orietta (Dr)",Earls Court Surgery GP,17407577,2022-07-14 10:00:00,21.6,0.0
2,05-Jul-22,14-Jul-22,Finished,10:00,"JAYATISSA, Rekha (Mrs)","EMILIANI, Orietta (Dr)",Earls Court Surgery GP,19579339,2022-07-14 10:00:00,21.6,0.0
3,05-Jul-22,14-Jul-22,Finished,10:00,"JAYATISSA, Rekha (Mrs)","EMILIANI, Orietta (Dr)",Earls Court Surgery GP,31978097,2022-07-14 10:00:00,21.6,0.0
4,11-Jul-22,14-Jul-22,Finished,10:00,"FRANKSON, Joyce (Ms)","EMILIANI, Orietta (Dr)",Earls Court Surgery GP,17407739,2022-07-14 10:00:00,21.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...
875730,18-Dec-19,19-Dec-19,Finished,19:00,Unknown,"THAXTER, James (Dr)",Redcliffe Extended Hours GP,47486862,2019-12-19 19:00:00,11.4,0.8
875731,18-Dec-19,19-Dec-19,Finished,19:00,"BISHOP, Jane (Reception Supervisor)","THAXTER, James (Dr)",Redcliffe Extended Hours GP,46949962,2019-12-19 19:00:00,11.4,0.8
875732,18-Dec-19,19-Dec-19,Finished,19:00,"BISHOP, Jane (Reception Supervisor)","THAXTER, James (Dr)",Redcliffe Extended Hours GP,47489972,2019-12-19 19:00:00,11.4,0.8
875733,19-Dec-19,19-Dec-19,Finished,19:00,"AMADI, Kasi (Miss)","THAXTER, James (Dr)",Redcliffe Extended Hours GP,21683033,2019-12-19 19:00:00,11.4,0.8


In [14]:
# Adding the function to merge the entire dataset available with weather dataset

def make_appointments_register(surgery_list = ['ECS', 'TCP', 'TGP', 'SMW', 'KMC', 'HPVM']):
    all_surgery_list = []
    
    for surgery in surgery_list:
        surgery_appointment_list = []
        
        for i in range(1,10):
            app_path = f'{RAW_DATA}{surgery}/{surgery}_app{i}.csv'
            single_app_file = pd.read_csv(app_path)
            surgery_appointment_list.append(single_app_file)
            
        surgery_apps_df = pd.concat(surgery_appointment_list, ignore_index=True)
        #print(f'The total rows of the {surgery} surgery is {surgery_apps_df.shape[0]}')
        
        all_surgery_list.append(surgery_apps_df)
        
    full_list = pd.concat(all_surgery_list, ignore_index=True)
    return full_list
make_appointments_register()

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842
1,12-May-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589
2,17-May-22,05-Jul-22,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135
3,31-May-22,07-Jul-22,Finished,11:00 - 11:59,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,19579593
4,07-Jun-22,05-Jul-22,Did Not Attend,11:00 - 11:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,60020944
...,...,...,...,...,...,...,...,...
877462,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47490926
877463,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47493202
877464,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47487575
877465,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"DUFFIELD, Katie (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47489391


In [20]:
df_copy = make_appointments_register()

In [24]:
# merging the final_dataset with weather csv file
df_weather['app datetime'] = pd.to_datetime(df_weather['app datetime'])

final_merged_df = pd.merge(final_dataset, df_weather, on='app datetime')
final_merged_df.head(-10)

KeyError: 'app datetime'

In [ ]:
import math

def haversine_distance(surgery_prefix, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers

    if surgery_prefix == 'ECS':
        lat1, lon1 = 51.488721, -0.191873
    elif surgery_prefix == 'SMW':
        lat1, lon1 = 51.494474, -0.181931
    elif surgery_prefix == 'TCP':
        lat1, lon1 = 51.48459, -0.171887
    elif surgery_prefix == 'HPVM':
        lat1, lon1 = 51.48459, -0.171887
    elif surgery_prefix == 'KMC':
        lat1, lon1 = 51.49807, -0.159918
    elif surgery_prefix == 'TGP':
        lat1, lon1 = 51.482652, -0.178066


    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance  # in kilometers

In [ ]:
def make_global_disease_register(surgery_list = ['ECS', 'TCP', 'TGP', 'SMW', 'KMC', 'HPVM']):
    print('=== Preparing Global Disease Register + IMD2023 info ======================================')

    disease_register = []
    for surgery in surgery_list:
        register_path = f'{RAW_DATA}{surgery}/{surgery}'

        idnhs = pd.read_excel(f'{register_path}_NHS_PTID.xlsx', dtype='str')
        idnhs.dropna(inplace=True)
        frail = pd.read_csv(f'{register_path}_FRAILTY.csv', dtype='str')
        dep = pd.read_csv(f'{register_path}_DEPRESSION.csv', dtype='str')
        obesity = pd.read_csv(f'{register_path}_OBESITY.csv', dtype='str')
        chd = pd.read_csv(f'{register_path}_IHD.csv', dtype='str')
        dm = pd.read_csv(f'{register_path}_DM.csv', dtype='str')
        hpt = pd.read_csv(f'{register_path}_HPT.csv', dtype='str')
        ndhg = pd.read_csv(f'{register_path}_NDHG.csv', dtype='str')
        smi = pd.read_csv(f'{register_path}_SMI.csv', dtype='str')

        ptid = idnhs.merge(frail, how='left', on='NHS number')
        ptid = ptid.drop(columns='NHS number')

        register = (ptid.merge(dep, how='left', on='Patient ID')
                    .merge(obesity, how='left', on='Patient ID')
                    .merge(chd, how='left', on='Patient ID')
                    .merge(dm, how='left', on='Patient ID')
                    .merge(hpt, how='left', on='Patient ID')
                    .merge(ndhg, how='left', on='Patient ID')
                    .merge(smi, how='left', on='Patient ID')
                    .fillna(0)
                    )
        print(f'💊 {surgery} Disease Register completed')
        # Add IMD and distance from station
        imd = pd.read_csv(IMD_DATA)

        full_register = register.merge(imd, how='left', on='Postcode')
        print(f'🔸 {surgery} IMD2023')
        full_register['distance_from_surg'] = full_register.apply(lambda row: haversine_distance(surgery, row['Latitude'], row['Longitude']), axis=1)
        disease_register.append(full_register)

    global_register = pd.concat(disease_register, axis=0, ignore_index=True)
    print(f"🦠 Concat Registers into ONE REGISTER")
    
    global_register.dropna(inplace=True)
    print(f'❌ Dropped NaN')
    
    output_path = f'{OUTPUT_DATA}global_disease_register.csv'
    global_register.to_csv(output_path, index=False)
    print(f'✅ Global Disease Register Saved to output-data: {global_register.shape}')
    print()
    return global_register

In [ ]:
disease_register = make_global_disease_register()